In [1]:
import pandas as pd
import numpy as np

In [2]:
sub_lgb = pd.read_csv('submissions/20190917_LightGBM_best_i_5_3_new_data.csv')
sub_cat = pd.read_csv('submissions/20190918_XGBoost_ft_eng_5_5_3.csv')
sub_xgb = pd.read_csv('submissions/20190918_XGBoost_ft_eng_5_5_3.csv')
sub = pd.read_csv('input/sample_submission.csv')

In [2]:
sub_r = pd.read_csv('submissions/20190830_new_ensemble_a_r_xgb_lgb_cat.csv')
sub_a = pd.read_csv('submissions/20190901_ensemble_A_R.csv')
sub = pd.read_csv('input/sample_submission.csv')

In [3]:
sub.head()

,TransactionID,isFraud
0,3663549,0.5
1,3663550,0.5
2,3663551,0.5
3,3663552,0.5
4,3663553,0.5


### MinMaxScaler

In [4]:
from sklearn.preprocessing import MinMaxScaler

In [5]:
scaler = MinMaxScaler()

In [11]:
sub_r_scaler = scaler.fit_transform(sub_r.isFraud.values.reshape(-1, 1))
sub_a_scaler = scaler.fit_transform(sub_a.isFraud.values.reshape(-1, 1))

### GMEAN

In [5]:
from scipy.stats import describe

In [6]:
concat_sub = pd.concat([sub_a.isFraud, sub_lgb.isFraud,
                        sub_xgb.isFraud, sub_cat.isFraud], axis=1)
concat_sub.shape

(506691, 4)

In [7]:
cols = list(map(lambda x: "m" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)

In [8]:
concat_sub.head()

,index,m0,m1,m2,m3
0,0,0.001828,0.000615,0.001021,0.000684
1,1,0.001557,0.002323,0.009134,0.001426
2,2,0.002441,0.001490,0.003796,0.001282
3,3,0.002791,0.003150,0.004202,0.002435
4,4,0.001758,0.001887,0.003232,0.001111


In [20]:
corr = concat_sub.iloc[:,1:].corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

In [21]:
rank = np.tril(concat_sub.iloc[:,1:].corr().values,-1)
m = (rank>0).sum()
m_gmean, s = 0, 0
for n in range(min(rank.shape[0],m)):
    mx = np.unravel_index(rank.argmin(), rank.shape)
    w = (m-n)/(m+n)
    print(w)
    m_gmean += w*(np.log(concat_sub.iloc[:,mx[0]+1])+np.log(concat_sub.iloc[:,mx[1]+1]))/2
    s += w
    rank[mx] = 1
m_gmean = np.exp(m_gmean/s)

1.0
0.7142857142857143
0.5
0.3333333333333333


In [22]:
describe(m_gmean)

DescribeResult(nobs=506691, minmax=(0.00018606186790794016, 0.9971642891582687), mean=0.02755998449777848, variance=0.012170791048464802, skewness=6.9078949859453225, kurtosis=49.87175829168274)

In [ ]:
sub['isFraud'] = m_gmean

In [22]:
sub.head(10)

,TransactionID,isFraud
0,3663549,0.001381
1,3663550,0.002337
2,3663551,0.002228
3,3663552,0.002962
4,3663553,0.001855
5,3663554,0.003734
6,3663555,0.012160
7,3663556,0.021180
8,3663557,0.000813
9,3663558,0.006604


In [9]:
sub.shape

(506691, 2)

In [24]:
sub.to_csv('submissions/benchmark_ensemble_gmean_ant_lgb_xgb_cat.csv', sep=',', header=True, index=None)

In [12]:
sub_r_scaler[:5]

array([[0.00050657],
       [0.00331102],
       [0.00170982],
       [0.00302157],
       [0.00180151]])

In [13]:
sub_a_scaler[:5]

array([[0.00168898],
       [0.00141766],
       [0.00230334],
       [0.00265393],
       [0.00161914]])

### Mean

In [10]:
preds = (sub_a.isFraud*sub_lgb.isFraud*
         sub_xgb.isFraud*sub_cat.isFraud)**(1/4)

In [4]:
preds = (sub_r.isFraud+sub_a.isFraud)/2

In [22]:
preds = (((sub_lgb.isFraud+sub_cat.isFraud)/2)+sub_xgb.isFraud)/2

In [ ]:
sub_lgb = pd.read_csv('submissions/benchmark_ft_selection_ft_eng_0_967_rounds.csv')
sub_xgb = pd.read_csv('submissions/benchmark_ft_selection_ft_eng_0_catboost.csv')

In [4]:
preds = (sub_lgb.isFraud+sub_xgb.isFraud+sub_cat.isFraud)/3

In [11]:
preds = (sub_r.isFraud+sub_a.isFraud)/2

In [12]:
# preds.head()

In [5]:
sub['isFraud'] = preds

In [6]:
sub.head(10)

,TransactionID,isFraud
0,3663549,0.001033
1,3663550,0.001967
2,3663551,0.002077
3,3663552,0.002617
4,3663553,0.001664
5,3663554,0.002726
6,3663555,0.014703
7,3663556,0.023018
8,3663557,0.001142
9,3663558,0.005859


In [7]:
sub.to_csv('submissions/20190918_ensemble_best_results.csv', sep=',', header=True, index=None)

### Ranking

In [8]:
sub_lgb = sub_lgb.sort_values('isFraud')
sub_cat = sub_cat.sort_values('isFraud')
sub_xgb = sub_xgb.sort_values('isFraud')

In [9]:
sub_cat.head()

,TransactionID,isFraud
113826,3777375,0.000011
427479,4091028,0.000011
133512,3797061,0.000012
453726,4117275,0.000012
95156,3758705,0.000012


In [10]:
sub_lgb['rank_l'] = list(range(len(sub_lgb)))
sub_cat['rank_c'] = list(range(len(sub_cat)))
sub_xgb['rank_x'] = list(range(len(sub_xgb)))

In [11]:
sub_cat.head()

,TransactionID,isFraud,rank_c
113826,3777375,0.000011,0
427479,4091028,0.000011,1
133512,3797061,0.000012,2
453726,4117275,0.000012,3
95156,3758705,0.000012,4


In [12]:
sub_lgb.head()

,TransactionID,isFraud,rank_l
275561,3939110,0.000026,0
245268,3908817,0.000028,1
176435,3839984,0.000029,2
186112,3849661,0.000029,3
379668,4043217,0.000031,4


In [13]:
df_sub = sub.join(sub_lgb[['TransactionID', 'rank_l']].set_index('TransactionID'), on=['TransactionID'], how='left')\
.join(sub_cat[['TransactionID', 'rank_c']].set_index('TransactionID'), on=['TransactionID'], how='left')\
.join(sub_xgb[['TransactionID', 'rank_x']].set_index('TransactionID'), on=['TransactionID'], how='left')

In [14]:
df_sub.head(10)

,TransactionID,isFraud,rank_l,rank_c,rank_x
0,3663549,0.000176,40741,19227,19227
1,3663550,0.000895,177644,213029,213029
2,3663551,0.000396,58885,137932,137932
3,3663552,0.000912,154837,234183,234183
4,3663553,0.000713,182174,144507,144507
5,3663554,0.000855,207719,163719,163719
6,3663555,0.005770,393200,398647,398647
7,3663556,0.014336,447775,449617,449617
8,3663557,0.000182,43057,20026,20026
9,3663558,0.001580,272419,259457,259457


In [15]:
df_sub['isFraud'] = (df_sub.rank_l + df_sub.rank_c + df_sub.rank_x) / 3
df_sub_final = df_sub[['TransactionID', 'isFraud']]
df_sub_final['isFraud'] = df_sub_final.isFraud / df_sub_final.isFraud.max()

/home/errodringer/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
df_sub_final.head(10)

,TransactionID,isFraud
0,3663549,0.052100
1,3663550,0.397157
2,3663551,0.220221
3,3663552,0.409986
4,3663553,0.309980
5,3663554,0.352064
6,3663555,0.783190
7,3663556,0.886156
8,3663557,0.054675
9,3663558,0.520594


In [17]:
df_sub_final.max()

TransactionID    4170239.0
isFraud                1.0
dtype: float64

In [18]:
df_sub_final.to_csv('submissions/20190918_ranking_ensemble_xgb_lgb_cat_ft_eng_5.csv', sep=',', header=True, index=None)